In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from google.colab import files
 
 
uploaded = files.upload()



Saving adult.csv to adult.csv


In [10]:
df = pd.read_csv('adult.csv')
df.head()



,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [6]:
df_missing = (df=='?').sum()
df_missing

age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
income               0
dtype: int64

In [37]:
df = df[df['workclass'] !='?']
X = df.drop('income', axis=1)
Y = df['income']

X

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
1,82,3,132870,11,9,6,4,1,4,0,0,4356,18,39
3,54,3,140359,5,4,0,7,4,4,0,0,3900,40,39
4,41,3,264663,15,10,5,10,3,4,0,0,3900,40,39
5,34,3,216864,11,9,0,8,4,4,0,0,3770,45,39
6,38,3,150601,0,6,5,1,4,4,1,0,3770,40,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,3,310152,15,10,4,11,1,4,1,0,0,40,39
32557,27,3,257302,7,12,2,13,5,4,0,0,0,38,39
32558,40,3,154374,11,9,2,7,0,4,1,0,0,40,39
32559,58,3,151910,11,9,6,1,4,4,0,0,0,40,39


In [9]:
from sklearn import preprocessing
df_categorical = df.select_dtypes(include=['object'])
df_categorical.head()

,workclass,education,marital.status,occupation,relationship,race,sex,native.country,income
1,Private,HS-grad,Widowed,Exec-managerial,Not-in-family,White,Female,United-States,<=50K
3,Private,7th-8th,Divorced,Machine-op-inspct,Unmarried,White,Female,United-States,<=50K
4,Private,Some-college,Separated,Prof-specialty,Own-child,White,Female,United-States,<=50K
5,Private,HS-grad,Divorced,Other-service,Unmarried,White,Female,United-States,<=50K
6,Private,10th,Separated,Adm-clerical,Unmarried,White,Male,United-States,<=50K


In [11]:
le = preprocessing.LabelEncoder()
df_categorical = df_categorical.apply(le.fit_transform)
df_categorical.head()

,workclass,education,marital.status,occupation,relationship,race,sex,native.country,income
1,3,11,6,4,1,4,0,39,0
3,3,5,0,7,4,4,0,39,0
4,3,15,5,10,3,4,0,39,0
5,3,11,0,8,4,4,0,39,0
6,3,0,5,1,4,4,1,39,0


In [24]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
ros.fit(X, Y)
X_resampled, Y_resampled = ros.fit_resample(X, Y)

In [49]:
from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=99)

In [50]:
from sklearn.preprocessing import LabelEncoder
for col in df.columns:
    if df[col].dtypes == 'object':
        encoder = LabelEncoder()
        df[col] = encoder.fit_transform(df[col])

In [51]:
X = df.drop('income', axis=1)
Y = df['income']

In [52]:
from sklearn.tree import DecisionTreeClassifier

# Fitting the decision tree with default hyperparameters, apart from
# max_depth which is 5 so that we can plot and read the tree.
dt_default = DecisionTreeClassifier(max_depth=5)
dt_default.fit(X_train,y_train)


DecisionTreeClassifier(max_depth=5)

In [53]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score


In [54]:
y_pred_default = dt_default.predict(X_test)
print(classification_report(y_test,y_pred_default))


              precision    recall  f1-score   support

           0       0.86      0.95      0.91      4644
           1       0.78      0.54      0.64      1501

    accuracy                           0.85      6145
   macro avg       0.82      0.74      0.77      6145
weighted avg       0.84      0.85      0.84      6145



In [55]:
print(confusion_matrix(y_test,y_pred_default))
print(accuracy_score(y_test,y_pred_default))

[[4413  231]
 [ 693  808]]
0.8496338486574451
